In [2]:
import scipy as sp
import numpy as np
import pandas as pd
import scanpy as sc


In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
# Function: save figure
def savefig(results_file, figid, aspect, exp_type, patients="All"):
    plt.tight_layout()
    plt.savefig(os.path.join(results_file, name_output(figid, patient_id=patients, aspect=aspect, exp_type=exp_type) + ".pdf"), dpi=600, bbox_inches='tight')
    plt.savefig(os.path.join(results_file, name_output(figid, patient_id=patients, aspect=aspect, exp_type=exp_type) + ".png"), dpi=600, bbox_inches='tight')



In [31]:
results_file = '/home/zengzw/tmp/figures/IGHV2-5--IGHJ6/'
figid = 1
patient_id = 'TNBC6'
sc.set_figure_params(fontsize=10)
adata = sc.read_10x_mtx(
        path = '/home/zengzw/HRA000477_outs/count/HRS122222_RNA/outs/filtered_feature_bc_matrix', var_names = 'gene_symbols', make_unique = True
        )
adata.var_names_make_unique()
adata.raw = adata

# Quality control
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Fig 1. Three violin plots
fig, axes = plt.subplots(1,3, figsize = (25,8))
plt.suptitle("Patient " + patient_id, verticalalignment = 'top', horizontalalignment = 'center', fontsize=25)
sc.pl.violin(adata, 'n_genes_by_counts', jitter=0.4, show=False, ax=axes[0])
sc.pl.violin(adata, 'total_counts', jitter=0.4, show=False, ax=axes[1])
sc.pl.violin(adata, 'pct_counts_mt', jitter=0.4, show=False, ax=axes[2])
savefig(results_file, figid, "violin_preQC", exp_type, patients = patient_id)
figid += 1

# Fig 2. Total vs genes
total_vs_genes = sns.jointplot(
    data=adata.obs,
    x="total_counts",
    y="n_genes_by_counts",
    kind="scatter",
    palette="blue"
)
plt.title("Patient " + patient_id, verticalalignment = 'bottom', horizontalalignment = 'left')
savefig(results_file, figid, "total_vs_genes", exp_type, patients = patient_id)
figid += 1

# Fig 3. Total vs mito.percentage
total_vs_mtcounts = sns.jointplot(
    data=adata.obs,
    x="total_counts",
    y="pct_counts_mt",
    kind="scatter",
    palette="blue"
)
plt.title("Patient " + patient_id, verticalalignment = 'bottom', horizontalalignment = 'left')
savefig(results_file, figid, "total_vs_mitocounts", exp_type, patients = patient_id)
figid += 1

# Doublet
counts_matrix = adata.X
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.1, sim_doublet_ratio=3)
scrub.call_doublets()
doublet_scores, predicted_doublets = scrub.scrub_doublets(n_prin_comps=50)

# Fig4: Doublet his
scrub.plot_histogram()
plt.suptitle("Patient " + patient_id, y = 0.999, x=0.55)
savefig(results_file, figid, "doublet_his", exp_type, patients = patient_id)
figid += 1

# Fig5: Doublet UMAP
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
scrub.plot_embedding('UMAP', order_points=True)
plt.suptitle("Patient " + patient_id, y = 0.999, x=0.55)
savefig(results_file, figid, "doublet_umap", exp_type, patients = patient_id)
figid += 1

total_counts= (adata.obs.total_counts >= 2000) & (adata.obs.total_counts <= 100000)
mt_outlier = (adata.obs.pct_counts_mt <= 10)
doublets = (predicted_doublets==False)

stat[i] = {
    "max_total_counts": max(adata.obs["total_counts"]), "min_total_counts": min(adata.obs["total_counts"]),
    "mean_total_counts": np.mean(adata.obs["total_counts"]), "medium_total_counts": np.median(adata.obs["total_counts"]),
    "max_features": max(adata.obs["n_genes_by_counts"]), "min_features": min(adata.obs["n_genes_by_counts"]),
    "mean_features": np.mean(adata.obs["n_genes_by_counts"]), "medium_features": np.median(adata.obs["n_genes_by_counts"]),
    "max_mito_counts": max(adata.obs["total_counts_mt"]), "min_mito_counts": min(adata.obs["total_counts_mt"]),
    "mean_mito_counts": np.mean(adata.obs["total_counts_mt"]), "medium_mito_counts": np.median(adata.obs["total_counts_mt"]),
    "max_mito_pct": max(adata.obs["pct_counts_mt"]), "min_mito_pct": min(adata.obs["pct_counts_mt"]),
    "mean_mito_pct": np.mean(adata.obs["pct_counts_mt"]), "medium_mito_pct": np.median(adata.obs["pct_counts_mt"]),
    "cell_number": len(adata.obs["total_counts"]), "gene_number": len(adata.var["gene_ids"]),
    "total_counts_less_than_2e3": sum(adata.obs.total_counts < 2000),
    "total_counts_more_than_1e5": sum(adata.obs.total_counts > 100000),
    "mito_outlier": len(adata.obs["total_counts"]) - np.sum(non_isoutlier(adata.obs.pct_counts_mt)[0]),
    "doublets": np.sum(predicted_doublets), "cell_number_afterQC": sum(total_counts & mt_outlier & doublets)
}

adata.obs["discard"] = total_counts & mt_outlier & doublets

# Fig 6 Total vs mito.percentage
total_vs_mtcounts = sns.jointplot(
    data=adata.obs,
    x="total_counts",
    y="pct_counts_mt",
    kind="scatter",
    hue=adata.obs.discard
)
plt.title("Patient " + patient_id, verticalalignment = 'bottom', horizontalalignment = 'left')
savefig(results_file, figid, "total_vs_mitocounts_afterQC", exp_type, patients = patient_id)
figid += 1

# After QC
adata = adata[adata.obs.discard, :]
stat[i]["mean_total_counts_afterQC"] = np.mean(adata.obs["total_counts"])
stat[i]["medium_total_counts_afterQC"] = np.median(adata.obs["total_counts"])
stat[i]["mean_mito_pct_afterQC"] = np.mean(adata.obs["pct_counts_mt"])
stat[i]["medium_mito_pct_afterQC"] = np.median(adata.obs["pct_counts_mt"])

# assert(type(adata_all.raw.X[0,0]) == np.int32)
# Normalization
sc.pp.normalize_total(adata, target_sum=1e4, key_added=True)
# Log_transform
sc.pp.log1p(adata)

# Feature selection
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
# Fig 7: Highly_variable genes
sc.pl.highly_variable_genes(adata, show=False)
savefig(results_file, figid, "filter_genes_dispersion", exp_type)

KeyboardInterrupt: 

In [17]:
adata_all.obs

,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,discard,True,batch,leiden,louvain,cell_type
AAACCTGAGAAGGCCT-1-TNBC01-BC,1149,2093.0,26.0,1.242236,True,2093.0,BC,7,8,NK cell
AAACCTGCAACACCCG-1-TNBC01-BC,1821,4384.0,110.0,2.509124,True,4384.0,BC,23,21,other
AAACCTGCAAGCTGGA-1-TNBC01-BC,1806,3833.0,26.0,0.678320,True,3833.0,BC,12,1,T cell
AAACCTGCAATGAAAC-1-TNBC01-BC,1565,2936.0,6.0,0.204360,True,2936.0,BC,7,8,NK cell
AAACCTGCACCAACCG-1-TNBC01-BC,1247,2277.0,30.0,1.317523,True,2277.0,BC,7,8,NK cell
...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTTACGTCA-1-GC10-Tumor-GC,1583,3567.0,305.0,8.550603,True,3567.0,GC,4,0,other
TTTGTCATCATTTGGG-1-GC10-Tumor-GC,1945,3053.0,114.0,3.734032,True,3053.0,GC,13,6,Tumor
TTTGTCATCGCCAGCA-1-GC10-Tumor-GC,4977,20114.0,728.0,3.619370,True,20114.0,GC,17,15,Fibro
TTTGTCATCTCGCTTG-1-GC10-Tumor-GC,2051,3264.0,37.0,1.133578,True,3264.0,GC,13,6,Tumor


In [20]:
sc.pl.highly_variable_genes(adata_all, show=False)
plt.savefig("/home/zengzw/tmp/tmp.png")

KeyError: 'hvg'